In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.externals import joblib 
import pickle

import tensorflow as tf
from keras.layers import *
from keras.regularizers import *
from keras.optimizers import *
from keras.models import *
from keras.preprocessing.text import *
from keras.callbacks import EarlyStopping
import keras.backend as K
from keras.utils.vis_utils import model_to_dot
from IPython.display import Image

import random
import os
random.seed(42)

/Users/apple/opt/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
Using TensorFlow backend.


### 데이터 전처리

In [2]:
new_df = pd.read_csv(os.path.join("..","..","data","input_data.csv"))
new_df.head()  

/Users/apple/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (4,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,user.id,place.id,place.name,category,place.category,user.rating,place.rating,createdDate,is_fch,photonum,review.count,language,lat,lng,land.addr,review,is_local
0,4TXey,1.150710e+09,스타벅스 보라매공원R점,EAT,스타벅스,4.5,4.590717,2020-05-21 20:42:00,1,0.0,158,ko,37.492103,126.92355,"서울특별시 동작구 신대방동 395-65 파크스퀘어,보라매현대APT",NaN,1
1,awWC,1.150710e+09,스타벅스 보라매공원R점,EAT,스타벅스,5.0,4.590717,2019-06-28 01:11:03,1,0.0,158,ko,37.492103,126.92355,"서울특별시 동작구 신대방동 395-65 파크스퀘어,보라매현대APT",none,1
2,1Unzg,1.150710e+09,스타벅스 보라매공원R점,EAT,스타벅스,5.0,4.590717,2020-03-12 00:22:00,1,0.0,158,ko,37.492103,126.92355,"서울특별시 동작구 신대방동 395-65 파크스퀘어,보라매현대APT",NaN,1
3,LRVl,1.150710e+09,스타벅스 보라매공원R점,EAT,스타벅스,5.0,4.590717,2020-02-16 17:06:00,1,0.0,158,ko,37.492103,126.92355,"서울특별시 동작구 신대방동 395-65 파크스퀘어,보라매현대APT",NaN,1
4,ESN1,1.150710e+09,스타벅스 보라매공원R점,EAT,스타벅스,5.0,4.590717,2019-03-11 00:08:48,1,0.0,158,ko,37.492103,126.92355,"서울특별시 동작구 신대방동 395-65 파크스퀘어,보라매현대APT",2019.03.06\nHH,1


In [3]:
new_df.rename(columns = {'place.id':'locationId'}, inplace=True)
new_df.rename(columns = {'place.rating':'rating'}, inplace=True)

f = new_df.groupby('locationId')['rating'].agg([('rated_count','count')]).reset_index()
new_df = pd.merge(new_df, f, on='locationId')

f = new_df.groupby('locationId')['photonum'].agg([('average_photonum','mean')]).reset_index()
new_df = pd.merge(new_df, f, on='locationId')

f = new_df.groupby('locationId')['rating'].agg([('average_rating','mean')]).reset_index()
new_df = pd.merge(new_df, f, on='locationId')

f = new_df.groupby('user.id')['rating'].agg([('user_mean_rating','mean')]).reset_index()
new_df = pd.merge(new_df, f, on='user.id')

f = new_df.groupby('user.id')['rating'].agg([('user_reviewcount','count')]).reset_index()
new_df = pd.merge(new_df, f, on='user.id')

d = new_df.groupby('user.id')['category'].agg([('a','count')]).reset_index()
d = d.reset_index()
d.rename(columns = {'index':'userID'}, inplace=True)
d.drop(columns='a', axis=1, inplace=True)

new_df = pd.merge(new_df, d, on='user.id')
new_df.drop(columns='user.id', axis=1, inplace=True)

new_df['category_l'] = new_df['category'].apply(lambda x: 0 if x =='EAT' else 1)
new_df['createdDate'] = new_df['createdDate'].apply(lambda x: x.split(' ')[0].replace('-',''))
new_df['locationId'] = new_df['locationId'].astype(int)
new_df['photonum'] = new_df['photonum'].astype(int)

new_df

,locationId,place.name,category,place.category,user.rating,rating,createdDate,is_fch,photonum,review.count,...,land.addr,review,is_local,rated_count,average_photonum,average_rating,user_mean_rating,user_reviewcount,userID,category_l
0,1150710065,스타벅스 보라매공원R점,EAT,스타벅스,4.5,4.590717,20200521,1,0,158,...,"서울특별시 동작구 신대방동 395-65 파크스퀘어,보라매현대APT",NaN,1,158,0.164557,4.590717,4.417995,105,42122,0
1,1219615644,해인어시장 신림본점,EAT,생선회-일반,2.5,4.525000,20200516,0,0,33,...,서울특별시 관악구 신림동 1433-120,NaN,1,33,0.151515,4.525000,4.417995,105,42122,0
2,13015318,보라매양평해장국,EAT,해장국-일반,5.0,4.445652,20200509,0,0,28,...,서울특별시 관악구 봉천동 972-4,NaN,1,28,0.035714,4.445652,4.417995,105,42122,0
3,31401167,배영만당진아구동태찜탕 보라매본점,EAT,"아귀찜,해물찜-일반",5.0,4.111111,20200508,0,0,24,...,서울특별시 관악구 봉천동 728-83,NaN,1,24,0.000000,4.111111,4.417995,105,42122,0
4,11784878,파리바게뜨 당곡사거리점,EAT,파리바게뜨,5.0,4.625000,20200508,1,0,26,...,서울특별시 관악구 봉천동 702-111,NaN,1,26,0.038462,4.625000,4.417995,105,42122,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472609,6998634,ibis budget Ambassador Seoul Dongdaemun,ACM,NaN,3.0,3.947230,20141121,0,6,379,...,NaN,We are a party of 8 and stayed for 4 nights.\n...,0,379,1.290237,3.947230,3.947230,1,31324,1
472610,6998634,ibis budget Ambassador Seoul Dongdaemun,ACM,NaN,4.0,3.947230,20141120,0,0,379,...,NaN,Positives: The location of this hotel is reall...,0,379,1.290237,3.947230,3.947230,1,67121,1
472611,6998634,ibis budget Ambassador Seoul Dongdaemun,ACM,NaN,4.0,3.947230,20141118,0,0,379,...,NaN,Location was great for shopping and also getti...,0,379,1.290237,3.947230,3.947230,1,46174,1
472612,6998634,ibis budget Ambassador Seoul Dongdaemun,ACM,NaN,4.0,3.947230,20141117,0,0,379,...,NaN,Great new hotel. Rooms were a bit small but wa...,0,379,1.290237,3.947230,3.947230,1,75850,1


In [4]:
new_df = new_df.loc[:,['locationId', 'place.name', 'category', 'rating', 'createdDate', 'is_fch', 'photonum', 
   'is_local', 'rated_count', 'average_photonum', 'average_rating', 'user_mean_rating',
       'user_reviewcount', 'userID', 'category_l','land.addr']]
new_df.head()

,locationId,place.name,category,rating,createdDate,is_fch,photonum,is_local,rated_count,average_photonum,average_rating,user_mean_rating,user_reviewcount,userID,category_l,land.addr
0,1150710065,스타벅스 보라매공원R점,EAT,4.590717,20200521,1,0,1,158,0.164557,4.590717,4.417995,105,42122,0,"서울특별시 동작구 신대방동 395-65 파크스퀘어,보라매현대APT"
1,1219615644,해인어시장 신림본점,EAT,4.525000,20200516,0,0,1,33,0.151515,4.525000,4.417995,105,42122,0,서울특별시 관악구 신림동 1433-120
2,13015318,보라매양평해장국,EAT,4.445652,20200509,0,0,1,28,0.035714,4.445652,4.417995,105,42122,0,서울특별시 관악구 봉천동 972-4
3,31401167,배영만당진아구동태찜탕 보라매본점,EAT,4.111111,20200508,0,0,1,24,0.000000,4.111111,4.417995,105,42122,0,서울특별시 관악구 봉천동 728-83
4,11784878,파리바게뜨 당곡사거리점,EAT,4.625000,20200508,1,0,1,26,0.038462,4.625000,4.417995,105,42122,0,서울특별시 관악구 봉천동 702-111


In [5]:
new_df.columns

Index(['locationId', 'place.name', 'category', 'rating', 'createdDate',
       'is_fch', 'photonum', 'is_local', 'rated_count', 'average_photonum',
       'average_rating', 'user_mean_rating', 'user_reviewcount', 'userID',
       'category_l', 'land.addr'],
      dtype='object')

In [6]:
new_df.shape[1]

16

In [7]:
dd = pd.read_json(os.path.join("..","..","data","TA_User_Review_Seoul_all_v2.json"))
dd.head()

,__typename,createdDate,helpfulVotes,id,language,location.__typename,location.additionalNames.long,location.additionalNames.longOnlyParent,location.locationId,location.name,...,socialStatistics.likeCount,socialStatistics.repostCount,socialStatistics.tripCount,text,title,userId,userProfile.contributionCounts.sumAllUgc,userProfile.isMe,userProfile.isVerified,userProfile.userId
0,Review,2020-03-06,0,749418557,ja,LocationInformation,"Hotel Gracery Seoul, South Korea",South Korea,14159911,Hotel Gracery Seoul,...,0,0,0,日系のホテルなので日本語OK\n南大門も明洞も歩いて行けます\nトリプルルーム お部屋は狭め...,グレイスリーホテルソウル,E44FFBC7D1E0A97FECCCF7F8EB052E0C,8,0,0,E44FFBC7D1E0A97FECCCF7F8EB052E0C
1,Review,2020-03-01,0,748454819,en,LocationInformation,"Hotel Gracery Seoul, South Korea",South Korea,14159911,Hotel Gracery Seoul,...,0,0,0,I really enjoyed our stay here in Gracery Hote...,Great Hotel,F9AD0C1F6409A1574AD0AFC237D2A4CE,7,0,0,F9AD0C1F6409A1574AD0AFC237D2A4CE
2,Review,2020-02-16,0,745371386,ja,LocationInformation,"Hotel Gracery Seoul, South Korea",South Korea,14159911,Hotel Gracery Seoul,...,0,0,0,今回はagoda経由で予約しました。予約後に日系のホテルと知った次第です。\nチェックイン後...,安心出来ます,770775D59B126E71D38D56FAB9B95F93,57,0,0,770775D59B126E71D38D56FAB9B95F93
3,Review,2020-02-11,0,744426202,ja,LocationInformation,"Hotel Gracery Seoul, South Korea",South Korea,14159911,Hotel Gracery Seoul,...,0,0,0,こちらのホテルは夫婦で何度か利用させていただいてます。立地もいいですが、お風呂とトイレが別な...,やはり落ち着く,4F1BF9403CA4EB27756E0C824A088553,43,0,0,4F1BF9403CA4EB27756E0C824A088553
4,Review,2020-02-05,0,743141310,en,LocationInformation,"Hotel Gracery Seoul, South Korea",South Korea,14159911,Hotel Gracery Seoul,...,0,0,0,Its about 7 minutes walking distance to City H...,"Ask for higher floor, it has better city view",23EF5413495FF530151DB7E5C0FEF38D,225,0,0,23EF5413495FF530151DB7E5C0FEF38D


In [8]:
dd = dd.loc[dd['location.placeType']=='EATERY'].loc[:,['userId','createdDate','location.locationId','location.name','location.placeType','rating','language','photoIds']]

dd.rename(columns = {'location.locationId':'locationId'}, inplace=True)
dd.rename(columns = {'location.name':'place.name'}, inplace=True)
dd.rename(columns = {'userId':'user.id'}, inplace=True)
dd.rename(columns = {'location.placeType':'category'}, inplace=True)
dd['photonum'] = dd['photoIds'].apply(lambda x: len(x))
dd['is_local'] = dd['language'].apply(lambda x: 1 if x=='ko' else 0)

f = dd.groupby('locationId')['rating'].agg([('rated_count','count')]).reset_index()
dd = pd.merge(dd, f, on='locationId')

f = dd.groupby('locationId')['photonum'].agg([('average_photonum','mean')]).reset_index()
dd = pd.merge(dd, f, on='locationId')

f = dd.groupby('locationId')['rating'].agg([('average_rating','mean')]).reset_index()
dd = pd.merge(dd, f, on='locationId')

f = dd.groupby('user.id')['rating'].agg([('user_mean_rating','mean')]).reset_index()
dd = pd.merge(dd, f, on='user.id')

f = dd.groupby('user.id')['rating'].agg([('user_reviewcount','count')]).reset_index()
dd = pd.merge(dd, f, on='user.id')

d = dd.groupby('user.id')['category'].agg([('a','count')]).reset_index()
d = d.reset_index()
d.rename(columns = {'index':'userID'}, inplace=True)
d.drop(columns='a', axis=1, inplace=True)

dd = pd.merge(dd, d, on='user.id')
dd.drop(columns='user.id', axis=1, inplace=True)

dd['category_l'] = dd['category'].apply(lambda x: 0 if x =='EAT' else 1)
dd['createdDate'] = dd['createdDate'].apply(lambda x: x.replace('-',''))
dd.drop(columns = 'photoIds', inplace=True)
dd.drop(columns = 'language', inplace=True)

dd

,createdDate,locationId,place.name,category,rating,photonum,is_local,rated_count,average_photonum,average_rating,user_mean_rating,user_reviewcount,userID,category_l
0,20180314,788542,Marco Polo,EATERY,5,0,1,20,0.050000,3.950000,4.363636,11,14256,1
1,20180226,9344928,Oasiseu,EATERY,4,0,1,11,1.636364,3.818182,4.363636,11,14256,1
2,20180309,8929037,Bongchu Braised Spicy Chicken With Vegetables,EATERY,4,0,1,5,0.200000,4.400000,4.363636,11,14256,1
3,20180309,9430705,Kkan Bu Chicken,EATERY,3,0,1,9,0.222222,3.777778,4.363636,11,14256,1
4,20151202,4032724,Yongdudong Jjukkumi,EATERY,4,0,1,12,0.000000,3.833333,4.363636,11,14256,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81640,20161021,10161356,MoMo Cafe,EATERY,5,0,1,120,0.500000,4.533333,5.000000,1,31337,1
81641,20161021,10161356,MoMo Cafe,EATERY,5,0,1,120,0.500000,4.533333,5.000000,1,8396,1
81642,20161016,10161356,MoMo Cafe,EATERY,5,0,1,120,0.500000,4.533333,5.000000,1,29116,1
81643,20161016,10161356,MoMo Cafe,EATERY,5,0,1,120,0.500000,4.533333,5.000000,1,30910,1


In [9]:
import pandas as pd

read_file = pd.read_excel(os.path.join("..","..","data",'TA_음식점_한글이름_주소_정보.xlsx'))

kr = read_file.loc[:,['location_id','name','address']]
kr.rename(columns = {'location_id':'locationId'}, inplace=True)
dd = pd.merge(kr, dd, on='locationId')
dd.drop(columns='place.name', axis=1, inplace=True)
dd['category'] = 'EAT'
dd.rename(columns = {'name':'place.name'}, inplace=True)
dd.rename(columns = {'address':'land.addr'}, inplace=True)
dd = dd.loc[:,['locationId', 'place.name', 'category', 'rating', 'createdDate',
        'photonum', 'is_local', 'rated_count', 'average_photonum',
       'average_rating', 'user_mean_rating', 'user_reviewcount', 'userID',
       'category_l', 'land.addr']]

print(dd.shape)
dd.head()

(81631, 15)


,locationId,place.name,category,rating,createdDate,photonum,is_local,rated_count,average_photonum,average_rating,user_mean_rating,user_reviewcount,userID,category_l,land.addr
0,788542,마르코 폴로,EAT,5,20180314,0,1,20,0.05,3.95,4.363636,11,14256,1,서울 강남구 삼성동 159-1 트레이드타워 52층
1,788542,마르코 폴로,EAT,4,20171207,0,1,20,0.05,3.95,4.000000,6,722,1,서울 강남구 삼성동 159-1 트레이드타워 52층
2,788542,마르코 폴로,EAT,5,20161110,0,0,20,0.05,3.95,5.000000,1,30372,1,서울 강남구 삼성동 159-1 트레이드타워 52층
3,788542,마르코 폴로,EAT,3,20160611,0,0,20,0.05,3.95,4.250000,4,2116,1,서울 강남구 삼성동 159-1 트레이드타워 52층
4,788542,마르코 폴로,EAT,4,20151204,0,1,20,0.05,3.95,4.095238,42,3208,1,서울 강남구 삼성동 159-1 트레이드타워 52층


In [10]:
dd_fch = read_file.loc[:,['name', 'web_url']]
dd_fch = dd_fch.drop_duplicates(['name'], keep='last')
dd_fch.rename(columns = {'name':'place.name'}, inplace=True)
dd_fch['is_fch'] = 0
dd_fch
# dd_fch.to_csv('dd_fch.csv', index=False)

,place.name,web_url,is_fch
0,마르코 폴로,https://www.tripadvisor.co.kr/Restaurant_Revie...,0
1,버터핑거 팬케이크 강남점,https://www.tripadvisor.co.kr/Restaurant_Revie...,0
2,빵꼼마,https://www.tripadvisor.co.kr/Restaurant_Revie...,0
3,명동한식당,https://www.tripadvisor.co.kr/Restaurant_Revie...,0
4,스타벅스 리져브 - 용산푸르지오써밋점,https://www.tripadvisor.co.kr/Restaurant_Revie...,0
...,...,...,...
5569,모모카페 코트야드 서울 남대문,https://www.tripadvisor.co.kr/Restaurant_Revie...,0
5570,스타벅스 경복궁역점,https://www.tripadvisor.co.kr/Restaurant_Revie...,0
5571,커피愛,https://www.tripadvisor.co.kr/Restaurant_Revie...,0
5572,해적선해물부페,https://www.tripadvisor.co.kr/Restaurant_Revie...,0


In [11]:
dd_fch = pd.read_excel(os.path.join("..","..","data",'dd_fch.xlsx'))
dd_fch

,place.name,web_url,is_fch
0,마르코 폴로,https://www.tripadvisor.co.kr/Restaurant_Revie...,0
1,버터핑거 팬케이크 강남점,https://www.tripadvisor.co.kr/Restaurant_Revie...,1
2,빵꼼마,https://www.tripadvisor.co.kr/Restaurant_Revie...,0
3,명동한식당,https://www.tripadvisor.co.kr/Restaurant_Revie...,0
4,스타벅스 리져브 - 용산푸르지오써밋점,https://www.tripadvisor.co.kr/Restaurant_Revie...,1
...,...,...,...
4768,모모카페 코트야드 서울 남대문,https://www.tripadvisor.co.kr/Restaurant_Revie...,0
4769,스타벅스 경복궁역점,https://www.tripadvisor.co.kr/Restaurant_Revie...,1
4770,커피愛,https://www.tripadvisor.co.kr/Restaurant_Revie...,1
4771,해적선해물부페,https://www.tripadvisor.co.kr/Restaurant_Revie...,0


In [12]:
dd = pd.merge(dd_fch, dd, on='place.name')
dd.head()

,place.name,web_url,is_fch,locationId,category,rating,createdDate,photonum,is_local,rated_count,average_photonum,average_rating,user_mean_rating,user_reviewcount,userID,category_l,land.addr
0,마르코 폴로,https://www.tripadvisor.co.kr/Restaurant_Revie...,0,788542,EAT,5,20180314,0,1,20,0.05,3.95,4.363636,11,14256,1,서울 강남구 삼성동 159-1 트레이드타워 52층
1,마르코 폴로,https://www.tripadvisor.co.kr/Restaurant_Revie...,0,788542,EAT,4,20171207,0,1,20,0.05,3.95,4.000000,6,722,1,서울 강남구 삼성동 159-1 트레이드타워 52층
2,마르코 폴로,https://www.tripadvisor.co.kr/Restaurant_Revie...,0,788542,EAT,5,20161110,0,0,20,0.05,3.95,5.000000,1,30372,1,서울 강남구 삼성동 159-1 트레이드타워 52층
3,마르코 폴로,https://www.tripadvisor.co.kr/Restaurant_Revie...,0,788542,EAT,3,20160611,0,0,20,0.05,3.95,4.250000,4,2116,1,서울 강남구 삼성동 159-1 트레이드타워 52층
4,마르코 폴로,https://www.tripadvisor.co.kr/Restaurant_Revie...,0,788542,EAT,4,20151204,0,1,20,0.05,3.95,4.095238,42,3208,1,서울 강남구 삼성동 159-1 트레이드타워 52층


In [13]:
# dd.drop(columns='web_url', axis=1, inplace=True)
dd = dd.loc[:,['locationId', 'place.name', 'category', 'rating', 'createdDate',
       'is_fch', 'photonum', 'is_local', 'rated_count', 'average_photonum',
       'average_rating', 'user_mean_rating', 'user_reviewcount', 'userID',
       'category_l', 'land.addr']]
dd.head()

,locationId,place.name,category,rating,createdDate,is_fch,photonum,is_local,rated_count,average_photonum,average_rating,user_mean_rating,user_reviewcount,userID,category_l,land.addr
0,788542,마르코 폴로,EAT,5,20180314,0,0,1,20,0.05,3.95,4.363636,11,14256,1,서울 강남구 삼성동 159-1 트레이드타워 52층
1,788542,마르코 폴로,EAT,4,20171207,0,0,1,20,0.05,3.95,4.000000,6,722,1,서울 강남구 삼성동 159-1 트레이드타워 52층
2,788542,마르코 폴로,EAT,5,20161110,0,0,0,20,0.05,3.95,5.000000,1,30372,1,서울 강남구 삼성동 159-1 트레이드타워 52층
3,788542,마르코 폴로,EAT,3,20160611,0,0,0,20,0.05,3.95,4.250000,4,2116,1,서울 강남구 삼성동 159-1 트레이드타워 52층
4,788542,마르코 폴로,EAT,4,20151204,0,0,1,20,0.05,3.95,4.095238,42,3208,1,서울 강남구 삼성동 159-1 트레이드타워 52층


In [14]:
dd.columns

Index(['locationId', 'place.name', 'category', 'rating', 'createdDate',
       'is_fch', 'photonum', 'is_local', 'rated_count', 'average_photonum',
       'average_rating', 'user_mean_rating', 'user_reviewcount', 'userID',
       'category_l', 'land.addr'],
      dtype='object')

In [15]:
dd.shape

(81011, 16)

In [16]:
new_df.columns

Index(['locationId', 'place.name', 'category', 'rating', 'createdDate',
       'is_fch', 'photonum', 'is_local', 'rated_count', 'average_photonum',
       'average_rating', 'user_mean_rating', 'user_reviewcount', 'userID',
       'category_l', 'land.addr'],
      dtype='object')

In [17]:
new_df.shape

(472614, 16)

In [18]:
final_df = pd.concat([dd, new_df])
print(final_df.shape)
final_df.head()

(553625, 16)


,locationId,place.name,category,rating,createdDate,is_fch,photonum,is_local,rated_count,average_photonum,average_rating,user_mean_rating,user_reviewcount,userID,category_l,land.addr
0,788542,마르코 폴로,EAT,5.0,20180314,0,0,1,20,0.05,3.95,4.363636,11,14256,1,서울 강남구 삼성동 159-1 트레이드타워 52층
1,788542,마르코 폴로,EAT,4.0,20171207,0,0,1,20,0.05,3.95,4.000000,6,722,1,서울 강남구 삼성동 159-1 트레이드타워 52층
2,788542,마르코 폴로,EAT,5.0,20161110,0,0,0,20,0.05,3.95,5.000000,1,30372,1,서울 강남구 삼성동 159-1 트레이드타워 52층
3,788542,마르코 폴로,EAT,3.0,20160611,0,0,0,20,0.05,3.95,4.250000,4,2116,1,서울 강남구 삼성동 159-1 트레이드타워 52층
4,788542,마르코 폴로,EAT,4.0,20151204,0,0,1,20,0.05,3.95,4.095238,42,3208,1,서울 강남구 삼성동 159-1 트레이드타워 52층


In [19]:
final_df.to_csv(os.path.join("..","..","data",'YN_final_df.csv'), index=False)


In [2]:
import pandas as pd
import os

new_df = pd.read_csv(os.path.join("..","..","data",'YN_final_df2.csv'))
new_df.head()

/Users/apple/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,rating,locationId,photonum,createdDate,is_fch,is_local,place.name,userID,average_photonum,average_rating,user_mean_rating,user_reviewcount,lat,lng,land.addr,category,rated_count,category_l
0,4.5,1150710065,0,2020-05-21 20:42:00,1,1,스타벅스 보라매공원R점,432293,0.164557,4.506329,4.5,1,37.492103,126.92355,"서울특별시 동작구 신대방동 395-65 파크스퀘어,보라매현대APT",EAT,158.0,1
1,5.0,1150710065,0,2019-06-28 01:11:03,1,1,스타벅스 보라매공원R점,135109,0.164557,4.506329,5.0,1,37.492103,126.92355,"서울특별시 동작구 신대방동 395-65 파크스퀘어,보라매현대APT",EAT,158.0,1
2,5.0,1150710065,0,2020-03-12 00:22:00,1,1,스타벅스 보라매공원R점,269138,0.164557,4.506329,5.0,1,37.492103,126.92355,"서울특별시 동작구 신대방동 395-65 파크스퀘어,보라매현대APT",EAT,158.0,1
3,5.0,1150710065,0,2020-02-16 17:06:00,1,1,스타벅스 보라매공원R점,246123,0.164557,4.506329,5.0,1,37.492103,126.92355,"서울특별시 동작구 신대방동 395-65 파크스퀘어,보라매현대APT",EAT,158.0,1
4,5.0,1150710065,0,2019-03-11 00:08:48,1,1,스타벅스 보라매공원R점,119248,0.164557,4.506329,5.0,1,37.492103,126.92355,"서울특별시 동작구 신대방동 395-65 파크스퀘어,보라매현대APT",EAT,158.0,1


In [4]:
new_df[~new_df['place.name'].str.contains("스타벅스|Korea")]

,rating,locationId,photonum,createdDate,is_fch,is_local,place.name,userID,average_photonum,average_rating,user_mean_rating,user_reviewcount,lat,lng,land.addr,category,rated_count,category_l
158,2.5,1219615644,0,2020-05-16 15:48:00,0,1,해인어시장 신림본점,439086,0.151515,4.424242,2.5,1,37.484363,126.926931,서울특별시 관악구 신림동 1433-120,EAT,33.0,1
159,4.0,1219615644,0,2019-09-13 14:05:00,0,1,해인어시장 신림본점,155415,0.151515,4.424242,4.0,1,37.484363,126.926931,서울특별시 관악구 신림동 1433-120,EAT,33.0,1
160,5.0,1219615644,0,2020-05-23 07:10:28,0,1,해인어시장 신림본점,432518,0.151515,4.424242,5.0,1,37.484363,126.926931,서울특별시 관악구 신림동 1433-120,EAT,33.0,1
161,5.0,1219615644,0,2020-04-24 21:28:00,0,1,해인어시장 신림본점,354570,0.151515,4.424242,5.0,1,37.484363,126.926931,서울특별시 관악구 신림동 1433-120,EAT,33.0,1
162,5.0,1219615644,0,2020-04-02 18:50:00,0,1,해인어시장 신림본점,316481,0.151515,4.424242,5.0,1,37.484363,126.926931,서울특별시 관악구 신림동 1433-120,EAT,33.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479686,2.0,8298647,2,2016-04-03,0,0,더스테이호텔,60218,2.000000,2.000000,2.0,1,37.561908,126.986790,서울특별시 중구 충무로2가 12-19,ACM,1.0,0
479687,2.0,2226404,0,2015-09-30,0,0,H2O 게스트하우스,43750,0.000000,2.000000,2.0,1,37.566925,126.995023,서울특별시 중구 산림동 293-1 석화사우나,ACM,1.0,0
479688,3.0,1625995,9,2015-09-22,0,0,봉 하우스,42921,9.000000,3.000000,3.0,1,37.584134,126.997032,서울특별시 종로구 명륜3가 117-2,ACM,1.0,0
479689,3.0,6892917,1,2013-08-06,0,0,24게스트하우스 신촌,25687,1.000000,3.000000,3.0,1,37.555017,126.935184,서울특별시 마포구 노고산동 57-20,ACM,1.0,0
